<a href="https://colab.research.google.com/github/XinzhouLi/Toxic_Language_Detection_in_Social_Media/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>